In [1]:
!pip install torch
!pip install git+https://github.com/openai/shap-e.git
!pip install pytorch3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [19]:
from shap_e.util.notebooks import decode_latent_mesh

In [15]:
print(torch.version.cuda)

12.4


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:14<00:00, 66.4MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [5]:
batch_size = 4
guidance_scale = 15.0
prompt = "desk"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [6]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAK58W6l0VKh0VKd0VKZ0VadzU6ZzU6ZzUqVzVKVzU6JzVKdyU6…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAKt0SatxPqpuM6ptLqdsPo1vYKdlGodqWYhpV4ZmVIFlV4NiTp…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAC8UCywVDSwUCi4TCi0TCiwTCi0SCi0SCSwSCiwSCSsSCisSCS…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAJpVG5dVGZZVGpZVGZZUGZZTG5VVGZVUGpVTG5VTGpVTGZRTGZ…

In [22]:
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
  image = decode_latent_mesh(xm, latent)
  display(image)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


TorchMesh(verts=tensor([[-0.9919, -0.9845,  0.4574],
        [-0.9922, -0.9845,  0.4729],
        [-0.9895, -0.9690,  0.4574],
        ...,
        [ 0.9845, -0.3023,  0.4807],
        [ 0.9845, -0.2868,  0.4692],
        [ 0.9845, -0.2868,  0.4762]], device='cuda:0'), faces=tensor([[33894, 14072,     0],
        [14072, 14073,     0],
        [14073,     1,     0],
        ...,
        [14071, 59091, 59089],
        [59090, 33893, 14071],
        [14071, 33893, 59091]], device='cuda:0'), vertex_channels={'R': tensor([0.3528, 0.4020, 0.3561,  ..., 0.3776, 0.3607, 0.3379], device='cuda:0'), 'G': tensor([0.1464, 0.1886, 0.1496,  ..., 0.1582, 0.1621, 0.1414], device='cuda:0'), 'B': tensor([0.0743, 0.1006, 0.0747,  ..., 0.0818, 0.0844, 0.0745], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.4612, -0.9845,  0.4884],
        [-0.4579, -0.9690,  0.4884],
        [-0.4574, -0.9535,  0.4884],
        ...,
        [ 0.4419,  0.9690,  0.5042],
        [ 0.4419,  0.9845,  0.4807],
        [ 0.4419,  0.9845,  0.5044]], device='cuda:0'), faces=tensor([[23246, 11642,     0],
        [11642, 23247,     0],
        [    0,     1, 23246],
        ...,
        [23244, 23245, 11640],
        [23245, 11641, 11640],
        [11641, 23245, 43743]], device='cuda:0'), vertex_channels={'R': tensor([0.1720, 0.1909, 0.1900,  ..., 0.2567, 0.1977, 0.2476], device='cuda:0'), 'G': tensor([0.0572, 0.0619, 0.0618,  ..., 0.0751, 0.0597, 0.0687], device='cuda:0'), 'B': tensor([0.0146, 0.0152, 0.0150,  ..., 0.0173, 0.0171, 0.0165], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.9906, -0.4729,  0.4884],
        [-0.9920, -0.4574,  0.4884],
        [-0.9857, -0.4574,  0.5039],
        ...,
        [ 0.9845,  0.4574,  0.5051],
        [ 0.9845,  0.4729,  0.4800],
        [ 0.9845,  0.4729,  0.5019]], device='cuda:0'), faces=tensor([[23238, 16012,     0],
        [16012, 23239,     0],
        [    0,     1, 23238],
        ...,
        [16010, 23236, 41745],
        [41746, 23237, 16011],
        [16011, 23237, 41747]], device='cuda:0'), vertex_channels={'R': tensor([0.0229, 0.0269, 0.0276,  ..., 0.0260, 0.0205, 0.0234], device='cuda:0'), 'G': tensor([0.0055, 0.0066, 0.0066,  ..., 0.0061, 0.0059, 0.0054], device='cuda:0'), 'B': tensor([0.0032, 0.0033, 0.0034,  ..., 0.0031, 0.0034, 0.0030], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.6350, -0.9845,  0.4729],
        [-0.6304, -0.9690,  0.4729],
        [-0.6292, -0.9535,  0.4729],
        ...,
        [ 0.6279,  0.9690,  0.4741],
        [ 0.6279,  0.9845,  0.4617],
        [ 0.6279,  0.9845,  0.4790]], device='cuda:0'), faces=tensor([[33834, 14998,     0],
        [14998, 33835,     0],
        [    0,     1, 33834],
        ...,
        [14997, 58699, 58697],
        [58698, 33833, 14997],
        [14997, 33833, 58699]], device='cuda:0'), vertex_channels={'R': tensor([0.1711, 0.2129, 0.2273,  ..., 0.2437, 0.2138, 0.2168], device='cuda:0'), 'G': tensor([0.0486, 0.0638, 0.0702,  ..., 0.0774, 0.0622, 0.0609], device='cuda:0'), 'B': tensor([0.0069, 0.0086, 0.0094,  ..., 0.0096, 0.0088, 0.0087], device='cuda:0')}, face_channels={})

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
